#### I`ll add more code examples and their results soon

In [ ]:
# Проверочный запуск
from pyspark.sql import SparkSession
import pyspark.pandas as sp

spark = SparkSession.builder.getOrCreate()
spark.sql("select 1").show()

In [ ]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark import SparkConf
import pyspark.pandas as ps

SPARK_CONFIG = [
    ('spark.driver.maxResultSize', '4g'),  # Должно быть не более 80% от памяти
    ('spark.driver.memory', '4g'), # Память на драйверах
    ('spark.executor.memory', '60g'), # Память на исполнителях
    ('spark.executor.memoryOverhead', '4g'), # Параметр допустимого переполнения памяти исполнителя
    ('spark.driver.memoryOverhead', '4g'), # Параметр допустимого переполнения памяти драйвера
    # Если включаешь динамическое добавление ресурсов, то отключи параметр ниже
    ('spark.executor.insatances', '5'), # Кол-во исполнителей, стоит учитывать, что драйвер и память будут выделены на инстант
    ('spark.executor.cores', '5'), #Было 8. Рекомендовано - 2, обычно ставят 5. Больше ядер - меньше исполнителей, меньше операций ввода-выводла и параллелизма
    ('spark.driver.cores', '5'), #Было 8. Рекомендовано - 2, обычно ставят 5. Больше ядер - меньше исполнителей, меньше операций ввода-выводла и параллелизма
    ('spark.cores.max', '15'), #Было 16, если ставим число не кратное  кол-ву исполнителей, получим висящие ядра, не участвующие в рабте
    ('spark.dynamicAllocation.enabled', 'false'), #Включить динамическое выделение ресурсов
    # ('spark.dynamicAllocation.minExecutors', '0'), #Дефолт - 1,то есть исполнители будут подключены автоматически при необходимости
    # ('spark.dynamicAllocation.maxExecutors', '24'), #Динамическое распределение исполнителей
    # ('spark.dynamicAllocation.executorIdleTimeout', '600s'), #Отключить исполнитель при простое более N-секунд (дефолт - 300)
    # ('spark.dynamicAllocation.cachedExecutorIdleTimeout', '600s'), #Отключить исполнитель с кэшем при простое более N-секунд (кэш храниться до падения сессии(дефолт - infinity))
    ('spark.sql.adaptive.enabled', 'True'),
    ('spark.sql.adaptive.forceApply', 'False'),
    ('spark.sql.adaptive.logLevel', 'info'),
    ('spark.sql.adaptive.advisoryPartitionSizeInBytes', '256m'),
    ('spark.sql.adaptive.coalescePartitions.enabled', 'True'),
    ('spark.sql.adaptive.coalescePartitions.minPartitionNum', '1'),
    ('spark.sql.adaptive.coalescePartitions.initialPartitionNum', '8192'),
    ('spark.sql.adaptive.fetchShuffleBlocksInBatch', 'True'),
    ('spark.sql.adaptive.localShuffleReader.enabled', 'True'),
    ('spark.sql.adaptive.skewJoin.enabled', 'True'),
    ('spark.sql.adaptive.skewJoin.skewedPartitionFactor', '5'),
    ('spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes', '400m'),
    ('spark.sql.adaptive.nonEmptyPartitionRatioForBroadcastJoin', '0.2'),
    ('spark.sql.autoBroadcastJoinThreshold', '-1'),
    ('spark.sql.optimizer.dynamicPartitionPrunning.enabled', 'True'),
    ('spark.sql.cbo.enabled', 'True'), # Включить Cost-based optimisation в Catalyst 
    ('spark.scheduler.mode', 'FAIR'), # Наиболее оптимальный алгоритм шедулера. Включать только в том случае, если он настроен
    ('spark.sql.codegen', 'false'), #Компилирует каждый запрос в байт-код Java на лету
    ('spark.sql.inMemoryColumnarStorage.compressed', 'False'), #Автоматическое сжатие табличного хранилища в памяти
    ('spark.sql.inMemoryColumnarStorage.batchsize', '1000'), #Размер пакета кэширования. Слишком большие значения приводят к ошибкам исчерпания памяти
    ('spark.sql.parquet.compression.codec', 'snappy'), #Используемый кодек сжатия (uncompressed, snappy, gzip, lzo)
    ('spark.sql.broadcastTimeout', 360), #5 минут таймаут ожидания трансляции данных (полезно, если broadcast готов не с самого начала работы запроса, а позднее)
    ('spark.sql.execution.arrow.enabled', 'true'), # Для эффективного преобразования датафрейма Spark в Pandas
    # ('spark.sql.session.timeZone', 'UTC+3'), #Устанавливаем локальный часовой пояс. Может быть конфликт с данными в Hive
    ('spark.bebug.maxToStringFields', '200'), #Максиммальное кол-во записей в строке дебага, все выходящее за кол-во будет скрыто
    ('spark.sql.parquet.binaryAsString', 'True'),
    ('spark.sql.parquet.int96TimestampConversion', 'True'),
    ('spark.sql.parquet.WriteLegacyFormat', 'True'),
    ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), #на всякий случай объявляем Kryo как механизм сериализации
    ('spark.kryoserializer.buffer', '24m'),
    ('spark.kryoserializer.buffer.max', '48m'), 
    ('spark.sparkContext.setLogLevel', 'OFF'), #- сократить размер логов с варнингами (ERROR\WARN)
    ('spark.sql.shuffle.partitions', '350'), # - Кол-во партиций для DataSet и DataFrame
    ('spark.default.parallelism', '350') # - Кол-во партиций для join и reduceByKey для RDD 
]

conf = SparkConf().setAll(SPARK_CONFIG)
spark = SparkSession.builder.config(conf = conf).getOrCreate()

#### To show spark general info like version etc. simply type the following

In [ ]:
spark

#### show Spark-config

In [ ]:
spark.sparkContext.getConf().getAll()

#### Show Spark-SQL query explain

In [ ]:
spark.sql('select * from tbl_nm').explain(extended = False)

####  Note that you can start a request either through an existing session or create a new one. Both options work pretty well

In [3]:
spark.sql("select count(*) from db_name.tbl_name").show()
# OR
spark.newSession().sql("select count(*) from db_name.tbl_name").show()

####  You could also make quries to **Hive** via **HiveSpark**

In [ ]:
from pyspark.sql import HiveContext

hiveCtx = HiveContext(sc)
# print(hiveCtx)
rows = hiveCtx.sql("select count(*) from db_name.tbl_name")
firstRow = rows.first()

firstRow

# Spark basic functions

#### Create DataFrame

In [ ]:
ds = [
    ['Миша', '111', 'Привет'],
    ['Жора', '111', 'Пока'],
    ['Глаша', '333', 'Как дела']
]
df = spark.createDataFrame(ds, ['col1', 'col2', 'col3'])

#### NOTE: Take into consideration that if the names of the columns are in <u>Russian</u>, then **df.filter('col1 like "Жора"').show()** will <u>NOT</u> work

#### To show dataframe schema

In [6]:
df.printSchema()
# OR
df.schema.fields

#### If you need to truncate DataFreame results  or show results vertically use:

In [10]:
df.show(truncate = 4)
#(100, truncate = False)  - show 100 rows and display full of data width, without cropping

# You could also show data vertically by:
df.show(truncate = False, vertical = True)

#### Calculate

In [ ]:
# rows
len(df.columns)

# Columns
df.count()

#### Select columns

In [ ]:
df.select('col1').show()

# If we want to show all columns 
df = df.select('*').show()

#### Functions

Documentation: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [ ]:
from pyspark.sql import functions as f

# You can simply port the col function so that you don’t have to write f.col every time
from pyspark.sql.functions import col

df.select('col1', f.concat('col2','col3').alias('конкатенированный')).toPandas()
# 'col1' can be replaced with df.columns[0], that is, refer to the column not by name, but by number

#### How to drop columns

df.drop('col1', 'col2').show()

#### How to add columns 

In [ ]:
# 1 способ - через withColumn
newdf = df.withColumn('Новый столбец', f.substring('col1', 1, 1))
newdf.show(1)

# 2 способ - через select
nefdf = df.select('*', f.substring('col1', 1, 1).alias('Новый столбец'))
newdf.show(1)

#### **filter()** and **distinct()**

In [ ]:
df.filter(f.col('col1').startswith('Жор')).show()
# OR
df.filter(f.col('col1').like('Жор%')).show()

# or HiveQL, remember about the column names in English
df.filter('col1 like "Жор%"').show()

#### <u>AND\OR</u>

In [14]:
# AND
df.filter('col1 like "Г%"').filter('col1 like "Ж%"').show()

# OR
df.filter('col1 like "Г%" or col1 like "Ж%"').show()

#Стоит запомнить, что побитовое ИЛИ по рангу выше операций сравнения, которые придется заключать в скобки
# StructuredAPI, побитовое ИЛИ - это как надо по хорошему
(
    df
    .filter(
        f.col('col1').startswith("Ж") | f.col('col1').startswith('Г')
    )
    .show()
)

#### Multiple 'Like'

In [ ]:
df. \
select(f.col('col_nm1'), f.col('col_nm2')). \
where('col_nm1 like "%НДС(НЕТ)%" or col_nm1 like "%БЕЗ(НДС%"'). \
show(truncate = False)

# OR

df. \
select(f.col('col_nm1'), f.col('col_nm2')). \
where(f.col('col_nm1').like('%НДС(НЕТ)%') | f.col('col_nm1').like('%БЕЗ(НДС%')). \
show(truncate = False)

#### Distinct

In [ ]:
df.select('col1').distinct().show()

#### JOIN and UNION

In [ ]:
# At firsst lest create 2 DataFrames
ctrydf = spark.createDataFrame(
    [
        [0, "ru", "Russia"],
        [1, "it", "Italy"],
        [2, "de", "Germany"]
    ],["id", "code", "name"]
)
ctrydf.show()

capitaldf = spark.createDataFrame(
        [
            [100, 'ru', 'Moscow', 140],
            [101, 'it', 'Rome', 100]
        ],['id', 'ctry_code', 'city', 'population']
)

capitaldf.show()

#### JOIN

In [ ]:
# You can use a joinexpression, in which we indicate how the tables will be linked
joinExpression = ctrydf['code'] == capitaldf['ctry_code']
ctrydf.join(capitaldf, joinExpression, 'inner').show()

# otherwise you would have to write it like this:
ctrydf.join(capitaldf, ctrydf['code'] == capitaldf['ctry_code'], 'inner').show()
